# 02 - Portfolios

In this notebook you can find simple examples that showcase how to programatically create portfolios.

In [1]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
%pip install seaborn

You should consider upgrading via the '/home/saheru/Documents/Programming/Finance/IBKR/ibeam/env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [2]:
import time
import logging

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [3]:
srv = Server()
srv.start()

# the following silences the logging output
# logging.getLogger('ibkr-algotrading').setLevel(logging.ERROR)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

First, it is a good idea to know which is our balance,

In [4]:
srv.balance

(1000000.0, 'USD')

> NOTE: with a simulated IBKR Paper Account you start with 1'000'000 USD, which you can use to test out trading.

## Create Portfolio

we can now request some stocks and create a portfolio with them,

In [5]:
stocks = srv[['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA'], '1y', '1d']

portfolio = Portfolio(wealth=100, stocks=stocks)

[2021-08-30 10:42:43,006] I T139720856172096 server.py:344: Started StockCache thread for bar "1d". 
		Next data update will be in  55036.99s.


The `wealth` argument is in the same currency as our account's balance.

We can now check how much fraction of stock is allocated for each of the stocks.

Of course, right now we haven't bought any stock so it's all zero,

In [6]:
portfolio.alloc

array([0., 0., 0., 0., 0.])

Similarly, our distribution of wealth accross the stocks looks like an all-zero array, and
this is the only time it will be like so before we perform our first order.

In any other case the "weights" will sum up to 1.

In [7]:
portfolio.weights

array([0., 0., 0., 0., 0.])

we finally add the portfolio to the server so it knows about its existance,

In [8]:
srv.add_portfolio(portfolio)

## Place an order on our portfolio

To prevent accidentally losing all of our money, we will trade under a simulated context in which orders are only previewed but we still get whole information on what would happen,

In [36]:
# distribute the portfolio equally accross stocks
weights = np.ones(5) / 5.0

with srv.simulated():
    # 1st order
    print('1st order')
    portfolio.order(weights)

    # the wealth of the portfolio decreases since the
    # transactions take a commission
    print('Portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 2nd order
    # we will put all our money on SPY
    weights[0] = 1.0
    weights[1:] = 0.0
    print('2nd order')
    portfolio.order(weights)

    print('Portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 3rd order
    # now we will give a litle to XOM
    weights[0] = 0.9
    weights[1] = 0.1
    print('3rd order')
    portfolio.order(weights)

    print('Portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # we can also list all orders that have happened on this portfolio.
    # the stored values are the weights, or distribution of wealth
    # accross the stocks. So, 0.2 on SPY means 20% of the portfolio's
    # wealth is in the form of SPY's shares at that given time.
    print('Portfolio orders:')
    print(portfolio.orders)

[2021-08-30 10:09:16,353] W T140319697635136 server.py:545: ENTERING SIMULATED CONTEXT. ALL FOLLOWING TRANSACTIONS WILL BE SIMULATED.


1st order


[2021-08-30 10:09:16,885] W T140318158599744 instruments.py:147: Provided quantity for stock "Stock("SPY", conid=756733)" is too precise. The following amount will be lost: 2.2768245622195593e-18
[2021-08-30 10:09:16,886] W T140318150207040 instruments.py:147: Provided quantity for stock "Stock("XOM", conid=13977)" is too precise. The following amount will be lost: 9.717161970901333e-18
[2021-08-30 10:09:16,887] W T140318166992448 instruments.py:147: Provided quantity for stock "Stock("MSFT", conid=272093)" is too precise. The following amount will be lost: 2.8053731213062427e-18
[2021-08-30 10:09:16,892] W T140318183777856 instruments.py:147: Provided quantity for stock "Stock("AAPL", conid=265598)" is too precise. The following amount will be lost: 9.999999999999544e-05
[2021-08-30 10:09:16,902] W T140318175385152 instruments.py:147: Provided quantity for stock "Stock("TSLA", conid=76792991)" is too precise. The following amount will be lost: 4.607859233063394e-19
[2021-08-30 10:09:1

Portfolio wealth:  99.02617500000001
commission: 0.9900990000000001 Wealth + commission:  100.01627400000001
2nd order


[2021-08-30 10:09:20,581] W T140318183777856 instruments.py:147: Provided quantity for stock "Stock("SPY", conid=756733)" is too precise. The following amount will be lost: 1.7076184216646695e-18
[2021-08-30 10:09:20,583] W T140318150207040 instruments.py:178: Provided quantity for stock "Stock("XOM", conid=13977)" is too precise. The following amount will be lost: 9.717161970901333e-18
[2021-08-30 10:09:20,584] W T140318175385152 instruments.py:178: Provided quantity for stock "Stock("MSFT", conid=272093)" is too precise. The following amount will be lost: 2.8053731213062427e-18
[2021-08-30 10:09:20,585] W T140318166992448 instruments.py:178: Provided quantity for stock "Stock("AAPL", conid=265598)" is too precise. The following amount will be lost: 9.999999999999544e-05
[2021-08-30 10:09:20,587] W T140318158599744 instruments.py:178: Provided quantity for stock "Stock("TSLA", conid=76792991)" is too precise. The following amount will be lost: 4.607859233063394e-19
[2021-08-30 10:09:2

Portfolio wealth:  98.06445000000001
commission: 1.9705561684132502 Wealth + commission:  100.03500616841326
3rd order


[2021-08-30 10:09:24,744] W T140318175385152 instruments.py:178: Provided quantity for stock "Stock("SPY", conid=756733)" is too precise. The following amount will be lost: 1.179069862577986e-18
[2021-08-30 10:09:24,745] W T140318158599744 instruments.py:147: Provided quantity for stock "Stock("XOM", conid=13977)" is too precise. The following amount will be lost: 9.999999999999641e-05
[2021-08-30 10:09:25,048] I T140319697635136 instruments.py:413: weights: [0.9 0.1 0.  0.  0. ] | diff: [-0.0237  0.1741  0.      0.      0.    ] | comm: ['0.11 USD' '0.10 USD'] | total spent: ['10.57 USD' '9.81 USD']
[2021-08-30 10:09:25,049] I T140319697635136 instruments.py:414: stk vals: [450.25  55.77 299.72 148.6  711.92] | share vals: ['10.68 USD (0.0237 SHARES)' '9.71 USD (0.1741 SHARES)']


Portfolio wealth:  97.10308200000001


[2021-08-30 10:09:26,523] W T140319697635136 server.py:552: LEAVING SIMULATED CONTEXT. ALL FOLLOWING TRANSACTIONS WILL BE REAL.


commission: 2.94149130721875 Wealth + commission:  100.04457330721877
Portfolio orders:
                            SPY  XOM  MSFT  AAPL  TSLA
2021-08-30 10:09:18.328948  0.2  0.2   0.2   0.2   0.2
2021-08-30 10:09:22.087066  1.0  0.0   0.0   0.0   0.0
2021-08-30 10:09:25.049764  0.9  0.1   0.0   0.0   0.0


Note that after we exit the simulated context, the whole state of the server will be restored, meaning all orders in the
portfolio will disappear and it will return to its previous state.

In [37]:
# after exiting the simulation, the portfolio does not have any orders
print('Portfolio orders:')
print(portfolio.orders)

Portfolio orders:
Empty DataFrame
Columns: []
Index: []


## Place an order back-in-time and check statistics

In a simulated context we can also "place orders" in the past by using historical data. This allows us to easily study the performance of the portfolio. 

> NOTE: back-in-time orders only work in a simulated context.

In [11]:
# we will use as order date one year before today
date = pd.Timestamp.today() - pd.Timedelta(days=365)
print('date of order:', date)

# distribute the portfolio equally accross stocks
weights = np.ones(5) / 5.0

with srv.simulated():
    # 1st order
    print('1st order')
    portfolio.order(weights, date=date)

    # the wealth of the portfolio decreases since the
    # transactions take a commission
    print('Today\'s portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Today\'s wealth + commission: ', portfolio.wealth + portfolio.commission)

    # we can also list all orders that have happened on this portfolio.
    # the stored values are the weights, or distribution of wealth
    # accross the stocks. So, 0.2 on SPY means 20% of the portfolio's
    # wealth is in the form of SPY's shares at that given time.
    print('Portfolio orders:')
    print(portfolio.orders)

[2021-08-30 10:43:33,835] W T139722359822144 server.py:547: ENTERING SIMULATED CONTEXT. ALL FOLLOWING TRANSACTIONS WILL BE SIMULATED.


date of order: 2020-08-30 10:43:33.834430
1st order


[2021-08-30 10:43:34,808] W T139720822601280 instruments.py:151: Provided quantity for stock "Stock("SPY", conid=756733)" is too precise. The following amount will be lost: 9.999999999999891e-05
[2021-08-30 10:43:34,808] W T139720830993984 instruments.py:151: Provided quantity for stock "Stock("XOM", conid=13977)" is too precise. The following amount will be lost: 9.999999999999818e-05
[2021-08-30 10:43:34,809] W T139720839386688 instruments.py:151: Provided quantity for stock "Stock("MSFT", conid=272093)" is too precise. The following amount will be lost: 3.2932640989247197e-18
[2021-08-30 10:43:34,810] W T139720847779392 instruments.py:151: Provided quantity for stock "Stock("AAPL", conid=265598)" is too precise. The following amount will be lost: 1.2197274440461925e-19
[2021-08-30 10:43:34,811] W T139720814208576 instruments.py:151: Provided quantity for stock "Stock("TSLA", conid=76792991)" is too precise. The following amount will be lost: 1.4772254600114998e-18
[2021-08-30 10:43:

Today's portfolio wealth:  134.401318


[2021-08-30 10:43:36,636] W T139722359822144 server.py:554: LEAVING SIMULATED CONTEXT. ALL FOLLOWING TRANSACTIONS WILL BE REAL.


commission: 0.9900990000000001 Today's wealth + commission:  135.391417
Portfolio orders:
                            SPY  XOM  MSFT  AAPL  TSLA
2020-08-30 10:43:33.834430  0.2  0.2   0.2   0.2   0.2
